# Binary classification benchmark

This notebook compares binary classification models from various libraries. The dataset used is the [Higgs dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS).

In [3]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,skgarden,sklearn,tensorflow,torch --datename

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Sat Sep 21 2019 

CPython 3.7.3
IPython 7.4.0

creme 0.3.0
keras 2.2.4
skgarden 0.1.2
sklearn 0.21.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.2.10-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [25]:
from creme import compose
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import neighbors
from creme import optim
from creme import preprocessing
from creme import stats
from keras import layers
from keras import models
from keras import optimizers
import numpy as np
import skgarden
from sklearn import linear_model as sk_linear_model
import torch

%run benchmarks/utils.py
%run benchmarks/wrappers.py

In [29]:
n_features = 28

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        self.sigmoid = torch.nn.Sigmoid()
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, activation='sigmoid', kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(), loss='binary_crossentropy')

converters = {name: float for name in names}
converters['is_signal'] = lambda x: bool(float(x))


import itertools

n = 500_000 # 11_000_000

get_X_y = lambda: itertools.islice(stream.iter_csv('HIGGS.csv.gz', names=names, target_name='is_signal', converters=converters), n)

results = benchmark(
    get_X_y=get_X_y,
    n=n,
    get_pp=preprocessing.StandardScaler,
    models=[
        ('creme', 'LogisticRegression', linear_model.LogisticRegression()),
        ('creme', 'PAClassifier', linear_model.PAClassifier()),
        ('scikit-learn', 'SGDClassifier', ScikitLearnClassifier(
            sk_linear_model.SGDClassifier(loss='log'),
            classes=[False, True]
        )),
        ('scikit-learn', 'PassiveAggressiveClassifier', ScikitLearnClassifier(
            sk_linear_model.PassiveAggressiveClassifier(),
            classes=[False, True]
        )),
        ('PyTorch (CPU)', 'Linear', PyTorchBinaryClassifier(
            network=torch_model,
            loss_fn=torch.nn.BCELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=0.01)
        )),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasBinaryClassifier(keras_model)),
        ('scikit-garden', 'MondrianTreeClassifier', ScikitLearnClassifier(
            skgarden.MondrianTreeClassifier(random_state=42),
            classes=np.array([False, True])
        )),
        ('scikit-garden', 'MondrianForestClassifier', ScikitLearnClassifier(
            skgarden.MondrianForestClassifier(random_state=42),
            classes=np.array([False, True])
        ))
    ],
    get_metric=metrics.Accuracy
)

In [27]:
results

,Library,Model,Accuracy,Fit time,Average fit time,Predict time,Average predict time
0,creme,LogisticRegression,0.609818,"254ms, 218μs, 897ns","23μs, 111ns","127ms, 655μs, 131ns","11μs, 605ns"
1,creme,PAClassifier,0.547909,"416ms, 951μs, 692ns","37μs, 905ns","256ms, 739μs, 604ns","23μs, 340ns"
2,scikit-learn,SGDClassifier,0.556636,"2s, 973ms, 296μs, 427ns","270μs, 300ns","817ms, 207μs, 712ns","74μs, 292ns"
3,scikit-learn,PassiveAggressiveClassifier,0.547909,"2s, 709ms, 773μs, 300ns","246μs, 343ns","769ms, 225μs, 668ns","69μs, 930ns"
4,PyTorch (CPU),Linear,0.609818,"2s, 422ms, 66μs, 2ns","220μs, 188ns","1s, 430ms, 21μs, 509ns","130μs, 2ns"
5,Keras on Tensorflow (CPU),Dense,0.609818,"5s, 190ms, 590μs, 567ns","471μs, 872ns","5s, 391ms, 946μs, 869ns","490μs, 177ns"
6,scikit-garden,MondrianTreeClassifier,0.543909,"6s, 959ms, 988μs, 118ns","632μs, 726ns","1s, 694ms, 959μs, 344ns","154μs, 87ns"
7,scikit-garden,MondrianForestClassifier,0.574455,"1m, 9s, 759ms, 552μs, 480ns","6ms, 341μs, 777ns","8s, 172ms, 406μs, 942ns","742μs, 946ns"
